In [1]:
! pip3 install chess
import subprocess
import os
from reti import utilities
import chess
import chess.engine
import chess.pgn
import itertools
import glob
import requests
import sys
import webbrowser


     |████████████████████████████████| 148 kB 7.6 MB/s eta 0:00:01


ModuleNotFoundError: No module named 'reti'

# Run CQL6 scripts on the pgns

In [16]:
cql_bin_dir = '/Users/elliottmacneil/chess/cql6/cql'
cql_scripts_dir = '/Users/elliottmacneil/python/reti/src/100endings/'
output_dir = '/Users/elliottmacneil/python/reti/output/output3/'
iterated = True
db_dir = '/Users/elliottmacneil/chess/pgn/twic/'

In [17]:
env = os.environ.copy()

def run_cql_scripts(cql_scripts_dir,db,count):
    for file in os.listdir(cql_scripts_dir):
        print(file)
        file_dir = cql_scripts_dir+'/'+file
        file_noext = os.path.splitext(file)[0]
        dir_noext = os.path.join(output_dir,file_noext)
        pgn_file = dir_noext+str(count)+'.pgn'
        cql_command = cql_bin_dir+' -i '+db+' -o '+pgn_file+' -matchcount 1 100 -threads 20 '+file_dir
        try:
          subprocess.run(cql_command,shell=True,env=env)
        except:
          print('failed')
        count += 1
    return 0

In [4]:
if iterated:
    count = 0
    for filename in os.listdir(db_dir):
        if filename.endswith('.pgn'):
            print(filename)
            db = db_dir + '/' + filename
            run_cql_scripts(cql_scripts_dir,db,count)
            count += 1
else:
    run_cql_scripts(cql_scripts_dir,db_dir,count=1)

twic_pgn_780_1160.pgn
99Qrp.cql
CQL version 6.1 (build 9.362) (c) Gady Costeff and Lewis Stiller
.........[10000]
.........[20000]
.........[30000]
.........[40000]
.........[50000]
.........[60000]
.........[70000]
.........[80000]
.........[90000]
.........[100000]
.........[110000]
.........[120000]
.........[130000]
.........[140000]
.........[150000]
.........[160000]
.........[170000]
.........[180000]
.........[190000]
.........[200000]
.........[210000]
.........[220000]
.........[230000]
.........[240000]
.........[250000]
.........[260000]
.........[270000]
.........[280000]
.........[290000]
.........[300000]
.........[310000]
.........[320000]
.........[330000]
.........[340000]
.........[350000]
.........[360000]
.........[370000]
.........[380000]
.........[390000]
.........[400000]
.........[410000]
.........[420000]
.........[430000]
.........[440000]
.........[450000]
.........[460000]
.........[470000]
.........[480000]
.........[490000]
.........[500000]
.........[51

# Process the PGN outputs from CQL6

In [23]:
def play_through_games(pgn_dir):
  content = utilities.load_pgn(pgn_dir)
  games = utilities.game_length_array(content)
  num_games = len(games)
  pgn = open(pgn_dir)
  for game in range(num_games):
    selected_game = chess.pgn.read_game(pgn)
    board = selected_game.board()
    for move in selected_game.main_line():
      board.push(move)
    print(board)
  return 0

In [ ]:
play_through_games('/content/drive/MyDrive/lichess_elite_db/elite_db_combined/lichess_elite_2020-06.pgn')

In [5]:
def print_relevant_positions(path):
    '''
    Given a path to a pgn file already processed by CQL6, print the relevant positions in the pgn file.
    '''
    fen_list = []
    pgn = open(path, encoding='utf-8', errors='replace')
    with open(path, encoding="utf-8",errors='replace') as file:
        content = file.read()
    num_games = utilities.game_length_array(content)[0]
    print('number of games:',num_games)
    for igame in range(num_games):
        game = chess.pgn.read_game(pgn)
        board = game.board()
        for node in game.mainline():
            move = node.move
            comment = node.comment 
            board.push(move)
            if comment == 'CQL':
                fens = board.fen()
                fen_list.append(fens)
                break
        else:
            continue
    return fen_list

In [19]:
# TODO given a file with fen strings, output the lichess practice from a position 

def fen2practice(path):
    fl = print_relevant_positions(path)
    for fen in fl:
        url = 'https://lichess.org/editor/'+fen.replace(' ', '_')
        webbrowser.open(url)
        
path = '/Users/elliottmacneil/python/reti/output/output3/93BN20.pgn'
fen2practice(path)

number of games: 37


In [ ]:
# write a tex file and execute it in latex
# images need to be in the same directory as the tex file i think 

fen2tex('test.tex', '/Users/elliottmacneil/python/reti/output/img')

In [ ]:
path = '/Users/elliottmacneil/python/reti/output/output3/1_5KP9.pgn'

fl = print_relevant_positions(path)

counter = 0
for fen in fl:
    print(fen)
    counter += 1
    filename = '/Users/elliottmacneil/python/reti/output/img/'+str(counter)+'.png'
    fen2png(fen,filename)
print(counter)

# Produce cql files looking for a given material imbalance

In [ ]:
def partition_pieces(num_white_pieces=5,num_black_pieces=5,diff=0):
  option = []
  piece_values = [9,5,3,1]
  white = list(itertools.combinations_with_replacement('9531',num_white_pieces))
  black = list(itertools.combinations_with_replacement('9531',num_black_pieces))

  white = [[int(x[i]) for i in range(len(x))] for x in white]
  black = [[int(x[i]) for i in range(len(x))] for x in black]

  white_final = []
  black_final = []
  for setup in white:
    if (setup.count(9) <= 1) and (setup.count(5) <= 2) and (setup.count(3) <= 4):
      white_final.append(setup)
  for setup in black:
    if (setup.count(9) <= 1) and (setup.count(5) <= 2) and (setup.count(3) <= 4):
      black_final.append(setup)

  for idx_w in range(len(white_final)):
    for idx_b in range(len(black_final)):
      if abs(sum(white_final[idx_w]) - sum(black_final[idx_b])) == diff:
        option.append([white_final[idx_w],black_final[idx_b]])
  return option

def values_to_pieces(partition):
  for opt_idx, option in enumerate(partition):
    for set_idx, setup in enumerate(option):
      for pc_idx, piece in enumerate(setup):
        if piece == 9:
          partition[opt_idx][set_idx][pc_idx] = 'Q'
          if set_idx == 1:
            partition[opt_idx][set_idx][pc_idx] = partition[opt_idx][set_idx][pc_idx].lower()
        elif piece == 5:
          partition[opt_idx][set_idx][pc_idx] = 'R'
          if set_idx == 1:
            partition[opt_idx][set_idx][pc_idx] = partition[opt_idx][set_idx][pc_idx].lower()
        elif piece == 3:
          partition[opt_idx][set_idx][pc_idx] = '[BN]'
          if set_idx == 1:
            partition[opt_idx][set_idx][pc_idx] = partition[opt_idx][set_idx][pc_idx].lower()
        elif piece == 1:
          partition[opt_idx][set_idx][pc_idx] = 'P'
          if set_idx == 1:
            partition[opt_idx][set_idx][pc_idx] = partition[opt_idx][set_idx][pc_idx].lower()
  return partition

In [ ]:
################################################################################
####################### TEST ###################################################

# ASSERT SYMMETRIC
for a in range(1,5):
  for b in range(1,5):
    for diff in range(5):
      assert len(partition_pieces(a,b,diff)) == len(partition_pieces(b,a,diff))

In [ ]:
part = partition_pieces(5,2,1)
values_to_pieces(part)

[[['Q', '[BN]', 'P', 'P', 'P'], ['q', 'r']],
 [['Q', 'P', 'P', 'P', 'P'], ['q', 'r']],
 [['Q', 'P', 'P', 'P', 'P'], ['q', '[bn]']],
 [['R', 'R', '[BN]', 'P', 'P'], ['q', 'r']],
 [['R', 'R', 'P', 'P', 'P'], ['q', 'r']],
 [['R', 'R', 'P', 'P', 'P'], ['q', '[bn]']],
 [['R', '[BN]', '[BN]', '[BN]', 'P'], ['q', 'r']],
 [['R', '[BN]', '[BN]', 'P', 'P'], ['q', 'r']],
 [['R', '[BN]', '[BN]', 'P', 'P'], ['q', '[bn]']],
 [['R', '[BN]', 'P', 'P', 'P'], ['q', '[bn]']],
 [['R', '[BN]', 'P', 'P', 'P'], ['q', 'p']],
 [['R', '[BN]', 'P', 'P', 'P'], ['r', 'r']],
 [['R', 'P', 'P', 'P', 'P'], ['q', 'p']],
 [['R', 'P', 'P', 'P', 'P'], ['r', 'r']],
 [['R', 'P', 'P', 'P', 'P'], ['r', '[bn]']],
 [['[BN]', '[BN]', '[BN]', '[BN]', 'P'], ['q', 'r']],
 [['[BN]', '[BN]', '[BN]', '[BN]', 'P'], ['q', '[bn]']],
 [['[BN]', '[BN]', '[BN]', 'P', 'P'], ['q', '[bn]']],
 [['[BN]', '[BN]', '[BN]', 'P', 'P'], ['q', 'p']],
 [['[BN]', '[BN]', '[BN]', 'P', 'P'], ['r', 'r']],
 [['[BN]', '[BN]', 'P', 'P', 'P'], ['q', 'p']],
 [['

In [ ]:
!rm -rf output
!mkdir output
%cd /content/drive/MyDrive/lichess_elite_db/elite_db_combined/reti/output/output


/content/drive/MyDrive/lichess_elite_db/elite_db_combined/reti/output/output


In [ ]:
def write_cql_files(white_pieces,black_pieces,value_diff):
  part = partition_pieces(white_pieces,black_pieces,value_diff)
  pieces = values_to_pieces(part)
  # [BN] seems to mean B or N.
  piece_key = ['Q','R','[BN]','P','q','r','[bn]','p']
  for piece_setup in pieces:
    piece_counts = []
    for idx in range(2):
      for key in piece_key:
        piece_counts.append(piece_setup[idx].count(key))
    with open('{}.cql'.format(''.join([''.join(x) for x in piece_setup])), 'a') as f:
      f.write('cql()\n')
      f.write('[Aa]={}\n'.format(white_pieces+black_pieces+2))
      for idx, key in enumerate(piece_key):
        f.write('{} == {}\n'.format(key,piece_counts[idx]))
  return 0


In [ ]:
write_cql_files(4,2,2)
!ls

'[BN][BN][bn][bn].cql'	   PPPPpp.cql		    'R[BN]PPr[bn].cql'
'[BN][BN][BN][BN]qp.cql'   PPPPrp.cql		    'R[BN]r[bn].cql'
'[BN][BN][BN][BN]qr.cql'  'Q[BN][BN]Pqr.cql'	    'RP[bn][bn].cql'
'[BN][BN][BN][BN]rr.cql'  'Q[BN]PPq[bn].cql'	    'RPPP[bn][bn].cql'
'[BN][BN][BN]Pq[bn].cql'  'Q[BN]q[bn].cql'	     RPPPqp.cql
'[BN][BN][BN]Pr[bn].cql'   QPPPqp.cql		     RPPPrp.cql
'[BN][BN]PP[bn][bn].cql'   QPPPqr.cql		     RPPPrr.cql
'[BN][BN]PPqp.cql'	   QPPPrr.cql		     RPrp.cql
'[BN][BN]PPrp.cql'	   QPqp.cql		    'RR[BN][BN]qr.cql'
'[BN][BN]PPrr.cql'	   QPrr.cql		    'RR[BN]Pq[bn].cql'
'[BN][BN]rp.cql'	   QRPPqr.cql		     RRPPqp.cql
'[BN]P[bn]p.cql'	   QRqr.cql		     RRPPqr.cql
'[BN]PPP[bn]p.cql'	  'R[BN][BN][BN]q[bn].cql'   RRPPrr.cql
'[BN]PPPr[bn].cql'	  'R[BN][BN]Pqp.cql'	     RRqp.cql
 output			  'R[BN][BN]Pqr.cql'	     RRrr.cql
'PPPP[bn][bn].cql'	  'R[BN][BN]Prr.cql'
 PPpp.cql		  'R[BN]PPq[bn].cql'
